# U.S. Bureau of Labor Statistics (BLS)

This example requests data on the number of job openings and hires in the construction industry from the BLS API.

In [1]:
# Preliminaries (Python 2.7)
import requests
import json
import config # file called config.py with my API key
import pandas as pd

import math

%matplotlib inline

### Multiple series, multiple decades

Even the version 2 API places limits on how many years worth of data can be returned in each request. For queries that require a longer time series, more than one request may be needed. 

This third example retrieves data on construction sector job openings and hires since 2000. 

In [2]:
# Include the start and end year here
date_range = (2015, 2017)

# Divide the date range into BLS-API-friendly length requests
req_no = int(math.ceil((date_range[1] - date_range[0]) / 10.0))
dates = []
for i in range(0,req_no):
    d1 = str(date_range[0]+i*10)
    d2 = str(date_range[0]+i*10+9)
    dates.append((d1,d2))
dates[-1] = (dates[-1][0], str(date_range[1]))

dates

[('2015', '2017')]

#### API and series information

In [3]:
# URL, key, and headers same as above
url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'
key = '?registrationkey={}'.format(config.bls_key)
headers = {'Content-type': 'application/json'}

# This time, store the series into a dictionary with description
series_dict = {'LNS11000006': 'Employment - Black',
               'LNS11000009': 'Employment - Hispanic',
               'LNS11000003': 'Employment - White',
               'LNS14000006': 'Unemp Rate - Black',
               'LNS14000009': 'Unemp Rate - Hispanic',
               'LNS14000003': 'Unemp Rate - White',
               'LNS11000018': 'Employment - Black - 16-19',
               'LNS11000021': 'Employment - Hispanic - 16-19',
               'LNS11000015': 'Employment - White - 16-19',
               'LNS14000018': 'Unemp Rate - Black - 16-19',
               'LNS14000021': 'Unemp Rate - Hispanic - 16-19',
               'LNS14000015': 'Unemp Rate - White - 16-19'}

df = pd.DataFrame()  # blank pandas dataframe to be filled later

#### Make requests for each date range in dates

In [4]:
for start, end in dates: 
    
    # The data sent in the post request now includes a start and end year
    data = json.dumps({"seriesid":series_dict.keys(), "startyear":start, "endyear":end})
    p = requests.post('{}{}'.format(url, key), headers=headers, data=data).json()
    d = {} # New dictionary to be filled with data
    for series in p['Results']['series']:
        s = series['seriesID']  # Shorten name to 's'
        
        # Add dictionary entry with series and reverse series order
        d[s] = pd.DataFrame(series['data']).iloc[::-1]
        if len(series['data']) > 0:  # This if/else is to allow for series of different lengths
            
            # Convert BLS API dates to readable format (YYYY-MM-DD)
            d[s]['date'] = pd.to_datetime(d[s]['period'] + ' ' + d[s]['year'])
            
            # Keep only date and series values
            d[s] = d[s].set_index('date')['value'].astype(float)
            
            # Rename and identify values as floating point numbers
            d[s] = d[s].rename(series_dict[s])
        else:  # If blank, leave as a blank pandas series
            d[s]['date'] = d[s]['value'] = pd.Series()
            d[s] = d[s]['value'].rename(series_dict[s])
            
    # Combine the dataframes for each range of years into one by appending        
    df = df.append(pd.concat([d[k] for k in series_dict.keys()], axis=1))

View the results

In [8]:
df.pct_change(24).tail(1)

,Unemp Rate - Hispanic,Employment - White - 16-19,Unemp Rate - White - 16-19,Unemp Rate - Black - 16-19,Unemp Rate - Black,Unemp Rate - White,Employment - Hispanic - 16-19,Employment - White,Employment - Black,Employment - Hispanic,Employment - Black - 16-19,Unemp Rate - Hispanic - 16-19
date,,,,,,,,,,,,
2017-05-01,-0.235294,-0.022479,-0.264516,-0.096026,-0.264706,-0.212766,0.108132,0.00555,0.03155,0.040729,0.105337,-0.256684
